<a href="https://colab.research.google.com/github/HarshitGupta1511/AI_ChatBot_NLP/blob/master/dwm_8th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

dataset = [['milk', 'bread', 'butter'],
           ['bread', 'butter'],
           ['milk', 'bread'],
           ['milk', 'butter'],
           ['bread']]

# One-hot encode
te = TransactionEncoder()
df = pd.DataFrame(te.fit(dataset).transform(dataset), columns=te.columns_)

# Generate frequent itemsets
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)
print("Apriori Frequent Itemsets:\n", frequent_itemsets)

# Generate rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
print("\nApriori Rules:\n", rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Apriori Frequent Itemsets:
    support         itemsets
0      0.8          (bread)
1      0.6         (butter)
2      0.6           (milk)
3      0.4  (butter, bread)
4      0.4    (milk, bread)
5      0.4   (milk, butter)

Apriori Rules:
   antecedents consequents  support  confidence      lift
0    (butter)     (bread)      0.4    0.666667  0.833333
1      (milk)     (bread)      0.4    0.666667  0.833333
2      (milk)    (butter)      0.4    0.666667  1.111111
3    (butter)      (milk)      0.4    0.666667  1.111111


ECLAT ALGORITHM

In [2]:
from itertools import combinations

data = [
    ['milk', 'bread', 'butter'],
    ['bread', 'butter'],
    ['milk', 'bread'],
    ['milk', 'butter'],
    ['bread']
]

# Convert to vertical format
vertical = {}
for tid, transaction in enumerate(data):
    for item in transaction:
        vertical.setdefault(item, set()).add(tid)

# ECLAT: Generate frequent 2-itemsets
min_support = 2
frequent = {}
for item1, item2 in combinations(vertical.keys(), 2):
    common_tids = vertical[item1] & vertical[item2]
    if len(common_tids) >= min_support:
        frequent[(item1, item2)] = len(common_tids)

print("ECLAT Frequent Itemsets:")
for itemset, support in frequent.items():
    print(f"{itemset}: {support}")


ECLAT Frequent Itemsets:
('milk', 'bread'): 2
('milk', 'butter'): 2
('bread', 'butter'): 2


FP GROWTH

In [3]:
from mlxtend.frequent_patterns import fpgrowth

# Reuse the one-hot encoded df from Apriori
frequent_itemsets_fp = fpgrowth(df, min_support=0.4, use_colnames=True)
print("FP-Growth Frequent Itemsets:\n", frequent_itemsets_fp)

rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.6)
print("\nFP-Growth Rules:\n", rules_fp[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


FP-Growth Frequent Itemsets:
    support         itemsets
0      0.8          (bread)
1      0.6           (milk)
2      0.6         (butter)
3      0.4    (milk, bread)
4      0.4   (milk, butter)
5      0.4  (butter, bread)

FP-Growth Rules:
   antecedents consequents  support  confidence      lift
0      (milk)     (bread)      0.4    0.666667  0.833333
1      (milk)    (butter)      0.4    0.666667  1.111111
2    (butter)      (milk)      0.4    0.666667  1.111111
3    (butter)     (bread)      0.4    0.666667  0.833333


FP TREE

In [4]:
class FPTreeNode:
    def __init__(self, item, count):
        self.item = item
        self.count = count
        self.children = {}

    def display(self, ind=1):
        print('  ' * ind, self.item, ' ', self.count)
        for child in self.children.values():
            child.display(ind + 1)

# Simple FP-Tree construction (not full FP-Growth logic)
root = FPTreeNode("Null", 1)

transactions = [['milk', 'bread'],
                ['bread', 'butter'],
                ['milk', 'bread', 'butter'],
                ['milk', 'butter']]

for transaction in transactions:
    current = root
    for item in transaction:
        if item in current.children:
            current.children[item].count += 1
        else:
            current.children[item] = FPTreeNode(item, 1)
        current = current.children[item]

print("FP-Tree:")
root.display()


FP-Tree:
   Null   1
     milk   3
       bread   2
         butter   1
       butter   1
     bread   1
       butter   1
